In [2]:
import joblib
import torch
import pandas as pd
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.2)
        
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.2)
        
        self.fc4 = nn.Linear(32, 3)
    
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        
        x = self.fc4(x)
        return x

# Check if MPS is available and set the device accordingly
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

print(f'Using device: {device}')

# Load the preprocessing pipeline
pipeline = joblib.load('preprocessing_pipeline_fine_tuned.pkl')

# Load the entire model
model = torch.load('model_fine_tuned.pth')
model.to(device)
model.eval()

# New data
song_features = pd.read_csv('extracted_features_to_predict.csv')

song_features


Using device: mps


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.942,0.476,10,-7.778,0,0.0499,0.191,0.000487,0.0949,0.721,120.984,audio_features,0NBkXEbOvbGyUgPHVdW3k1,spotify:track:0NBkXEbOvbGyUgPHVdW3k1,https://api.spotify.com/v1/tracks/0NBkXEbOvbGy...,https://api.spotify.com/v1/audio-analysis/0NBk...,164262,4


In [3]:
# Transform new data using the same pipeline
song_transformed = pipeline.transform(song_features)

# Convert to tensor and move to device
song_tensor = torch.tensor(song_transformed, dtype=torch.float32).to(device)

# Make predictions
with torch.no_grad():
    predictions = model(song_tensor)
    _, predicted_classes = torch.max(predictions, 1)

# Convert tensor to list and print (.numpy not working since lib conflict)
result = predicted_classes.cpu().tolist()
if result == [0]:
    print('Low chance getting award')
elif result == [1]:
    print('Possible getting award')
elif result == [2]:
    print('High chance getting award')

High chance getting award
